# Day1-1 실습
https://github.com/FranxYao/chain-of-thought-hub/blob/main/gsm8k/gpt3.5turbo_gsm8k_complex.ipynb

In [1]:
# !pip install openai
# !pip install datasets

In [2]:
from openai import OpenAI
import re
import time

import numpy as np

from tqdm import tqdm
from datasets import load_dataset

In [3]:
# api_key from file
file_path = 'openai_api.txt'

with open(file_path, 'r') as f:
    api_key = f.read().strip()

In [4]:
client = OpenAI(
    api_key=api_key,
)

In [5]:
gsm8k = load_dataset('gsm8k', 'main')
validation_index = np.load('lib_prompt/validation_index.npy')
validation_data = gsm8k['train'].select(validation_index)
gsm8k_test = gsm8k['test']

In [6]:
gsm8k['train'][0]['question']

'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?'

In [7]:
gsm8k_test = gsm8k['test']

In [8]:
prompt_complex = open('lib_prompt/prompt_hardest.txt').read()

In [9]:
print(prompt_complex)

Question: Angelo and Melanie want to plan how many hours over the next week they should study together for their test next week. They have 2 chapters of their textbook to study and 4 worksheets to memorize. They figure out that they should dedicate 3 hours to each chapter of their textbook and 1.5 hours for each worksheet. If they plan to study no more than 4 hours each day, how many days should they plan to study total over the next week if they take a 10-minute break every hour, include 3 10-minute snack breaks each day, and 30 minutes for lunch each day?
Let's think step by step
Angelo and Melanie think they should dedicate 3 hours to each of the 2 chapters, 3 hours x 2 chapters = 6 hours total.
For the worksheets they plan to dedicate 1.5 hours for each worksheet, 1.5 hours x 4 worksheets = 6 hours total.
Angelo and Melanie need to start with planning 12 hours to study, at 4 hours a day, 12 / 4 = 3 days.
However, they need to include time for breaks and lunch. Every hour they want 

In [10]:
prompt_complex

"Question: Angelo and Melanie want to plan how many hours over the next week they should study together for their test next week. They have 2 chapters of their textbook to study and 4 worksheets to memorize. They figure out that they should dedicate 3 hours to each chapter of their textbook and 1.5 hours for each worksheet. If they plan to study no more than 4 hours each day, how many days should they plan to study total over the next week if they take a 10-minute break every hour, include 3 10-minute snack breaks each day, and 30 minutes for lunch each day?\nLet's think step by step\nAngelo and Melanie think they should dedicate 3 hours to each of the 2 chapters, 3 hours x 2 chapters = 6 hours total.\nFor the worksheets they plan to dedicate 1.5 hours for each worksheet, 1.5 hours x 4 worksheets = 6 hours total.\nAngelo and Melanie need to start with planning 12 hours to study, at 4 hours a day, 12 / 4 = 3 days.\nHowever, they need to include time for breaks and lunch. Every hour they

In [11]:
prompt_q = prompt_complex + '\nQuestion: ' + gsm8k_test[1]['question'] + '\n'

In [12]:
print(prompt_q)

Question: Angelo and Melanie want to plan how many hours over the next week they should study together for their test next week. They have 2 chapters of their textbook to study and 4 worksheets to memorize. They figure out that they should dedicate 3 hours to each chapter of their textbook and 1.5 hours for each worksheet. If they plan to study no more than 4 hours each day, how many days should they plan to study total over the next week if they take a 10-minute break every hour, include 3 10-minute snack breaks each day, and 30 minutes for lunch each day?
Let's think step by step
Angelo and Melanie think they should dedicate 3 hours to each of the 2 chapters, 3 hours x 2 chapters = 6 hours total.
For the worksheets they plan to dedicate 1.5 hours for each worksheet, 1.5 hours x 4 worksheets = 6 hours total.
Angelo and Melanie need to start with planning 12 hours to study, at 4 hours a day, 12 / 4 = 3 days.
However, they need to include time for breaks and lunch. Every hour they want 

In [13]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "Follow the given examples and answer the question."},
        {"role": "user", "content": prompt_q},
    ]
)

In [14]:
response.choices[0].message.content

'To calculate the total number of bolts required for the robe, we start by considering the blue fiber bolts needed. Given that the robe takes 2 bolts of blue fiber and half that much white fiber, we find:\n\nBlue fiber bolts needed: 2 bolts\n\nSince the robe requires half as many white fiber bolts as blue fiber bolts, the number of white fiber bolts needed is half of 2, which equals:\n\nWhite fiber bolts needed: 2 bolts / 2 = 1 bolt\n\nTherefore, to find the total number of bolts required for the robe, we add the blue fiber bolts and white fiber bolts:\n\nTotal bolts needed: 2 blue fiber bolts + 1 white fiber bolt = 3 bolts\n\nSo, the robe takes a total of 3 bolts of fiber.'

In [15]:
print(response.choices[0].message.content)

To calculate the total number of bolts required for the robe, we start by considering the blue fiber bolts needed. Given that the robe takes 2 bolts of blue fiber and half that much white fiber, we find:

Blue fiber bolts needed: 2 bolts

Since the robe requires half as many white fiber bolts as blue fiber bolts, the number of white fiber bolts needed is half of 2, which equals:

White fiber bolts needed: 2 bolts / 2 = 1 bolt

Therefore, to find the total number of bolts required for the robe, we add the blue fiber bolts and white fiber bolts:

Total bolts needed: 2 blue fiber bolts + 1 white fiber bolt = 3 bolts

So, the robe takes a total of 3 bolts of fiber.


## LLMlinguage2 를 사용하여 300 token target 으로 compress 후 compress 하지 않은 경우와 결과 비교
https://github.com/microsoft/LLMLingua

In [16]:
# !pip install llmlingua

In [17]:
from llmlingua import PromptCompressor

llm_lingua = PromptCompressor(device_map="cpu") # 맥북 로컬 환경에서는 cpu 사용하도록 설정. 안하면 CUDA 오류남. AssertionError: Torch not compiled with CUDA enabled
compressed_prompt = llm_lingua.compress_prompt(prompt_complex, target_token=300)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
compressed_prompt

{'compressed_prompt': ': Angelo andanie to plan many hours study their test2 of4 to They should study study total week they1 hour,3 breaks and\ns think step\nanie eachters hours hours\nate1 hours each hours\n andanie to start with planning study,\n to. they to203ute\nunch for  minutes hour3\nSo andanie study study,5 to for\nThe4\n\n: Marks team52.  score teams\ns,5\n team also63, they8\n and as point\n together0\ns scoreds scored4 half57\nThe is The\n: B has two as than deck she2 more what of have she\n\n haveella manybles as0\n Bella25 will\n more deck, have\n2+0The\n4 fruitanas first2 many there\n\n\n,8\nets are3anges theThe3 basket4\nThe1\n:13ruits between cost\n\n, then62\n1If\n of is and price\n can)2\n\n:y a large,0,, many the most\n\n,0y0 total\n,:0 Heanged into\n boxestersbox0He50ese three to3, 1000 - 480 = 520 do not like to play basketball.\nThe percentage of the school that do not like to play basketball is 520/1000 * 100 = 52\nThe answer is 52\n',
 'origin_tokens': 2365,
 '

In [21]:
compressed_prompt["compressed_prompt"]

': Angelo andanie to plan many hours study their test2 of4 to They should study study total week they1 hour,3 breaks and\ns think step\nanie eachters hours hours\nate1 hours each hours\n andanie to start with planning study,\n to. they to203ute\nunch for  minutes hour3\nSo andanie study study,5 to for\nThe4\n\n: Marks team52.  score teams\ns,5\n team also63, they8\n and as point\n together0\ns scoreds scored4 half57\nThe is The\n: B has two as than deck she2 more what of have she\n\n haveella manybles as0\n Bella25 will\n more deck, have\n2+0The\n4 fruitanas first2 many there\n\n\n,8\nets are3anges theThe3 basket4\nThe1\n:13ruits between cost\n\n, then62\n1If\n of is and price\n can)2\n\n:y a large,0,, many the most\n\n,0y0 total\n,:0 Heanged into\n boxestersbox0He50ese three to3, 1000 - 480 = 520 do not like to play basketball.\nThe percentage of the school that do not like to play basketball is 520/1000 * 100 = 52\nThe answer is 52\n'

In [22]:
compressed_prompt_q = compressed_prompt["compressed_prompt"] + '\nQuestion: ' + gsm8k_test[1]['question'] + '\n'

In [23]:
compressed_prompt_q

': Angelo andanie to plan many hours study their test2 of4 to They should study study total week they1 hour,3 breaks and\ns think step\nanie eachters hours hours\nate1 hours each hours\n andanie to start with planning study,\n to. they to203ute\nunch for  minutes hour3\nSo andanie study study,5 to for\nThe4\n\n: Marks team52.  score teams\ns,5\n team also63, they8\n and as point\n together0\ns scoreds scored4 half57\nThe is The\n: B has two as than deck she2 more what of have she\n\n haveella manybles as0\n Bella25 will\n more deck, have\n2+0The\n4 fruitanas first2 many there\n\n\n,8\nets are3anges theThe3 basket4\nThe1\n:13ruits between cost\n\n, then62\n1If\n of is and price\n can)2\n\n:y a large,0,, many the most\n\n,0y0 total\n,:0 Heanged into\n boxestersbox0He50ese three to3, 1000 - 480 = 520 do not like to play basketball.\nThe percentage of the school that do not like to play basketball is 520/1000 * 100 = 52\nThe answer is 52\n\nQuestion: A robe takes 2 bolts of blue fiber and 

In [24]:
print(compressed_prompt_q)

: Angelo andanie to plan many hours study their test2 of4 to They should study study total week they1 hour,3 breaks and
s think step
anie eachters hours hours
ate1 hours each hours
 andanie to start with planning study,
 to. they to203ute
unch for  minutes hour3
So andanie study study,5 to for
The4

: Marks team52.  score teams
s,5
 team also63, they8
 and as point
 together0
s scoreds scored4 half57
The is The
: B has two as than deck she2 more what of have she

 haveella manybles as0
 Bella25 will
 more deck, have
2+0The
4 fruitanas first2 many there


,8
ets are3anges theThe3 basket4
The1
:13ruits between cost

, then62
1If
 of is and price
 can)2

:y a large,0,, many the most

,0y0 total
,:0 Heanged into
 boxestersbox0He50ese three to3, 1000 - 480 = 520 do not like to play basketball.
The percentage of the school that do not like to play basketball is 520/1000 * 100 = 52
The answer is 52

Question: A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts i

In [25]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "Follow the given examples and answer the question."},
        {"role": "user", "content": compressed_prompt_q},
    ]
)

In [26]:
response.choices[0].message.content

'Total bolts needed = 2 (blue) + 1 (half of blue for white) = 3 bolts.\nIt takes a total of 3 bolts to make the robe.'